In [ ]:
import os
import binascii

# Path to the memory dump (adjust to your file)
dump_file = r"C:\Users\xxxxx\Documents\minidump.dmp"

# PNG header and footer (IEND chunk)
PNG_HEADER = bytes.fromhex("89504E470D0A1A0A")
PNG_FOOTER = bytes.fromhex("49454E44AE426082")

def extract_pngs(file_path):
    # Output folder for extracted PNGs
    base_output_dir = os.path.join(os.path.dirname(file_path), "extracted_pngs")
    output_dir = base_output_dir
    counter = 1
    while os.path.exists(output_dir):
        output_dir = f"{base_output_dir}_{counter}"
        counter += 1
    os.makedirs(output_dir, exist_ok=True)

    # File name for logging
    filename = os.path.basename(file_path)
    print(f"Scanning: {filename}")

    # Read the file in binary mode
    with open(file_path, "rb") as f:
        data = f.read()

        # Find all occurrences of the PNG header
        start_positions = [i for i in range(len(data) - len(PNG_HEADER)) 
                           if data[i:i+len(PNG_HEADER)] == PNG_HEADER]

        if not start_positions:
            print(f"  - No PNG found in {filename}")
            return

        # For each found header, search for the footer
        for idx, start in enumerate(start_positions):
            end_pos = data.find(PNG_FOOTER, start)
            if end_pos != -1:
                end_pos += len(PNG_FOOTER)  # Footer ends after IEND
                png_data = data[start:end_pos]

                # Save the extracted PNG
                output_file = os.path.join(output_dir, f"extracted_{idx}.png")
                with open(output_file, "wb") as png_file:
                    png_file.write(png_data)
                print(f"  - Found and saved PNG: {output_file}")
            else:
                print(f"  - PNG header found at position {start}, but missing IEND")

if __name__ == "__main__":
    print("Starting PNG search in memory dump...")
    if os.path.isfile(dump_file):
        extract_pngs(dump_file)
    else:
        print(f"Error: File {dump_file} does not exist!")
    print("Scanning completed.")